In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image

In [6]:
df = pd.read_csv('full_df.csv', index_col=0)

In [7]:
df.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
ID,,,,,,,,,,,,,,,,,,
0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [8]:
# Load and preprocess the data
def load_data(df, image_dir, img_size=(224, 224)):
    images = []
    labels = []
    for _, row in df.iterrows():
        for side in ['Left-Fundus', 'Right-Fundus']:
            img_path = os.path.join(image_dir, row[side])
            if os.path.exists(img_path):
                # Load and preprocess image
                img = Image.open(img_path).resize(img_size).convert('RGB')
                images.append(np.array(img) / 255.0)  # Normalize

                # Append labels
                labels.append(eval(row['target']))
    return np.array(images), np.array(labels)

In [9]:
# Define file paths and parameters
image_dir = "C:\\Users\\ihsan\\OneDrive\\Masaüstü\\Kaggle\\Canon Ocular Disease Recognition\\ODIR-5K\\Training Images"
img_size = (224, 224)

In [10]:
# Load data
X, y = load_data(df, image_dir, img_size)

In [11]:
# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y.shape[1], activation='sigmoid')  # Multilabel classification
])

In [13]:
# Compile the model
with tf.device('/GPU:0'):
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [14]:
# Train the model
callbacks = [EarlyStopping(patience=5, restore_best_weights=True)]
with tf.device('/GPU:0'):
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20,
        batch_size=32,
        callbacks=callbacks
    )

Epoch 1/20
320/320 [==============================] - 135s 417ms/step - loss: 0.3198 - accuracy: 0.4359 - val_loss: 0.2964 - val_accuracy: 0.4451
Epoch 2/20
320/320 [==============================] - 127s 397ms/step - loss: 0.3004 - accuracy: 0.4523 - val_loss: 0.2953 - val_accuracy: 0.4400
Epoch 3/20
320/320 [==============================] - 125s 390ms/step - loss: 0.2957 - accuracy: 0.4531 - val_loss: 0.2883 - val_accuracy: 0.4439
Epoch 4/20
320/320 [==============================] - 124s 389ms/step - loss: 0.2897 - accuracy: 0.4579 - val_loss: 0.2833 - val_accuracy: 0.4400
Epoch 5/20
320/320 [==============================] - 124s 387ms/step - loss: 0.2863 - accuracy: 0.4610 - val_loss: 0.2821 - val_accuracy: 0.4583
Epoch 6/20
320/320 [==============================] - 126s 394ms/step - loss: 0.2810 - accuracy: 0.4743 - val_loss: 0.2773 - val_accuracy: 0.4627
Epoch 7/20
320/320 [==============================] - 123s 383ms/step - loss: 0.2775 - accuracy: 0.4825 - val_loss: 0.2823 -

In [15]:
# Save the model
model.save('fundus_disease_model.h5')